In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
import pickle
import wandb
from wandb.integration.xgboost import WandbCallback
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import gaussian_kde
from sklearn.preprocessing import StandardScaler

In [ ]:
print(list(np.around(np.linspace(0.01,0.15,5),2)))


In [ ]:
print([i for i in range(3,7)],)

In [ ]:
regression_data=pd.read_csv(r"C:\Users\asenlen\Desktop\Projects\tez\modeller\lending_club\regression_data.csv")
regression_data.isnull().sum()
regression_data.notnull().shape

In [ ]:
regression_data=regression_data[regression_data['mort_acc'].notnull()]
regression_data.shape
regression_data.isnull().sum()

In [ ]:
regression_data.shape

In [ ]:
regression_data['loan_status'],loan_uniques=pd.factorize(regression_data['loan_status'], sort=True)
regression_data['loan_status']=1-regression_data['loan_status'].astype("int")
regression_data['loan_status'].value_counts()

In [ ]:
debt_consolidation_regression=regression_data[regression_data['purpose']=='debt_consolidation'].reset_index(drop=True)
credit_card_regression=regression_data[regression_data['purpose']=='credit_card'].reset_index(drop=True)
home_improvement_regression=regression_data[regression_data['purpose']=='home_improvement'].reset_index(drop=True)
other_regression=regression_data[regression_data['purpose']=='other'].reset_index(drop=True)
major_purchase_regression=regression_data[regression_data['purpose']=='major_purchase'].reset_index(drop=True)
small_business_regression=regression_data[regression_data['purpose']=='small_business'].reset_index(drop=True)
medical_regression=regression_data[regression_data['purpose']=='medical'].reset_index(drop=True)
car_regression=regression_data[regression_data['purpose']=='car'].reset_index(drop=True)
moving_regression=regression_data[regression_data['purpose']=='moving'].reset_index(drop=True)
vacation_regression=regression_data[regression_data['purpose']=='vacation'].reset_index(drop=True)
house_regression=regression_data[regression_data['purpose']=='house'].reset_index(drop=True)
wedding=regression_data[regression_data['purpose']=='wedding'].reset_index(drop=True)
renewable_energy_regression=regression_data[regression_data['purpose']=='renewable_energy'].reset_index(drop=True)
educational_regression=regression_data[regression_data['purpose']=='educational'].reset_index(drop=True)

In [ ]:
regression_data=None

In [ ]:
#model denemeleri 
from sklearn.model_selection import train_test_split
debt_consolidation_regression_train, debt_consolidation_regression_test = train_test_split(debt_consolidation_regression, test_size=0.20, random_state=42)
credit_card_regression_train, credit_card_regression_test = train_test_split(credit_card_regression, test_size=0.20, random_state=42)
home_improvement_regression_train, home_improvement_regression_test = train_test_split(home_improvement_regression, test_size=0.20, random_state=42)
other_regression_train, other_regression_test = train_test_split(other_regression, test_size=0.20, random_state=42)
major_purchase_regression_train, major_purchase_regression_test = train_test_split(major_purchase_regression, test_size=0.20, random_state=42)
small_business_regression_train, small_business_regression_test = train_test_split(small_business_regression, test_size=0.20, random_state=42)
medical_regression_train, medical_regression_test = train_test_split(medical_regression, test_size=0.20, random_state=42)
car_regression_train, car_regression_test = train_test_split(car_regression, test_size=0.20, random_state=42)
moving_regression_train, moving_regression_test = train_test_split(moving_regression, test_size=0.20, random_state=42)
vacation_regression_train, vacation_regression_test = train_test_split(vacation_regression, test_size=0.20, random_state=42)
house_regression_train, house_regression_test = train_test_split(house_regression, test_size=0.20, random_state=42)
wedding_train, wedding_test = train_test_split(wedding, test_size=0.20, random_state=42)
renewable_energy_regression_train, renewable_energy_regression_test = train_test_split(renewable_energy_regression, test_size=0.20, random_state=42)

In [ ]:
debt_consolidation_regression_y,debt_consolidation_regression_x=debt_consolidation_regression['loan_status'],debt_consolidation_regression.drop(['purpose','loan_status'],axis=1)

In [ ]:
debt_consolidation_regression_train_y,debt_consolidation_regression_train_x=debt_consolidation_regression_train['loan_status'],debt_consolidation_regression_train.drop(['purpose','loan_status'],axis=1)
debt_consolidation_regression_test_y,debt_consolidation_regression_test_x=debt_consolidation_regression_test['loan_status'],debt_consolidation_regression_test.drop(['purpose','loan_status'],axis=1)

credit_card_regression_train_y,credit_card_regression_train_x=credit_card_regression_train['loan_status'],credit_card_regression_train.drop(['purpose','loan_status'],axis=1)
credit_card_regression_test_y,credit_card_regression_test_x=credit_card_regression_test['loan_status'],credit_card_regression_test.drop(['purpose','loan_status'],axis=1)

home_improvement_regression_train_y,home_improvement_regression_train_x=home_improvement_regression_train['loan_status'],home_improvement_regression_train.drop(['purpose','loan_status'],axis=1)
home_improvement_regression_test_y,home_improvement_regression_test_x=home_improvement_regression_test['loan_status'],home_improvement_regression_test.drop(['purpose','loan_status'],axis=1)

other_regression_train_y,other_regression_train_x=other_regression_train['loan_status'],other_regression_train.drop(['purpose','loan_status'],axis=1)
other_regression_test_y,other_regression_test_x=other_regression_test['loan_status'],other_regression_test.drop(['purpose','loan_status'],axis=1)

major_purchase_regression_train_y,major_purchase_regression_train_x=major_purchase_regression_train['loan_status'],major_purchase_regression_train.drop(['purpose','loan_status'],axis=1)
major_purchase_regression_test_y,major_purchase_regression_test_x=major_purchase_regression_test['loan_status'],major_purchase_regression_test.drop(['purpose','loan_status'],axis=1)

small_business_regression_train_y,small_business_regression_train_x=small_business_regression_train['loan_status'],small_business_regression_train.drop(['purpose','loan_status'],axis=1)
small_business_regression_test_y,small_business_regression_test_x=small_business_regression_test['loan_status'],small_business_regression_test.drop(['purpose','loan_status'],axis=1)

medical_regression_train_y,medical_regression_train_x=medical_regression_train['loan_status'],medical_regression_train.drop(['purpose','loan_status'],axis=1)
medical_regression_test_y,medical_regression_test_x=medical_regression_test['loan_status'],medical_regression_test.drop(['purpose','loan_status'],axis=1)

car_regression_train_y,car_regression_train_x=car_regression_train['loan_status'],car_regression_train.drop(['purpose','loan_status'],axis=1)
car_regression_test_y,car_regression_test_x=car_regression_test['loan_status'],car_regression_test.drop(['purpose','loan_status'],axis=1)

moving_regression_train_y,moving_regression_train_x=moving_regression_train['loan_status'],moving_regression_train.drop(['purpose','loan_status'],axis=1)
moving_regression_test_y,moving_regression_test_x=moving_regression_test['loan_status'],moving_regression_test.drop(['purpose','loan_status'],axis=1)

vacation_regression_train_y,vacation_regression_train_x=vacation_regression_train['loan_status'],vacation_regression_train.drop(['purpose','loan_status'],axis=1)
vacation_regression_test_y,vacation_regression_test_x=vacation_regression_test['loan_status'],vacation_regression_test.drop(['purpose','loan_status'],axis=1)

house_regression_train_y,house_regression_train_x=house_regression_train['loan_status'],house_regression_train.drop(['purpose','loan_status'],axis=1)
house_regression_test_y,house_regression_test_x=house_regression_test['loan_status'],house_regression_test.drop(['purpose','loan_status'],axis=1)

wedding_train_y,wedding_train_x=wedding_train['loan_status'],wedding_train.drop(['purpose','loan_status'],axis=1)
wedding_test_y,wedding_test_x=wedding_test['loan_status'],wedding_test.drop(['purpose','loan_status'],axis=1)

renewable_energy_regression_train_y,renewable_energy_regression_train_x=renewable_energy_regression_train['loan_status'],renewable_energy_regression_train.drop(['purpose','loan_status'],axis=1)
renewable_energy_regression_test_y,renewable_energy_regression_test_x=renewable_energy_regression_test['loan_status'],renewable_energy_regression_test.drop(['purpose','loan_status'],axis=1)

In [ ]:
debt_consolidation_regression_train_x.shape

In [ ]:
amcik_sote=resample(debt_consolidation_regression_train,n_samples=300,replace=True)
print(amcik_sote[amcik_sote["loan_status"]==0].shape[0])
print(amcik_sote[amcik_sote["loan_status"]==1].shape[0])

In [ ]:
amin_evladi={}
amin_evladi["d"]=[]
amin_evladi["d"].append(3)
amin_evladi

In [ ]:
def kl_mvn(m0, S0, m1, S1):
    """
    https://stackoverflow.com/questions/44549369/kullback-leibler-divergence-from-gaussian-pm-pv-to-gaussian-qm-qv
    The following function computes the KL-Divergence between any two 
    multivariate normal distributions 
    (no need for the covariance matrices to be diagonal)
    Kullback-Liebler divergence from Gaussian pm,pv to Gaussian qm,qv.
    Also computes KL divergence from a single Gaussian pm,pv to a set
    of Gaussians qm,qv.
    Diagonal covariances are assumed.  Divergence is expressed in nats.
    - accepts stacks of means, but only one S0 and S1
    From wikipedia
    KL( (m0, S0) || (m1, S1))
         = .5 * ( tr(S1^{-1} S0) + log |S1|/|S0| + 
                  (m1 - m0)^T S1^{-1} (m1 - m0) - N )
    # 'diagonal' is [1, 2, 3, 4]
    tf.diag(diagonal) ==> [[1, 0, 0, 0]
                          [0, 2, 0, 0]
                          [0, 0, 3, 0]
                          [0, 0, 0, 4]]
    # See wikipedia on KL divergence special case.              
    #KL = 0.5 * tf.reduce_sum(1 + t_log_var - K.square(t_mean) - K.exp(t_log_var), axis=1)   
                if METHOD['name'] == 'kl_pen':
                self.tflam = tf.placeholder(tf.float32, None, 'lambda')
                kl = tf.distributions.kl_divergence(oldpi, pi)
                self.kl_mean = tf.reduce_mean(kl)
                self.aloss = -(tf.reduce_mean(surr - self.tflam * kl))                               
    """
    # store inv diag covariance of S1 and diff between means
    N = m0.shape[0]
    iS1 = np.linalg.inv(S1)
    diff = m1 - m0

    # kl is made of three terms
    tr_term   = np.trace(iS1 @ S0)
    det_term  = np.log(np.linalg.det(S1)/np.linalg.det(S0)) #np.sum(np.log(S1)) - np.sum(np.log(S0))
    quad_term = diff.T @ np.linalg.inv(S1) @ diff #np.sum( (diff*diff) * iS1, axis=1)
    #print(tr_term,det_term,quad_term)
    return .5 * (tr_term + det_term + quad_term - N)

In [ ]:
(debt_consolidation_regression_train_y.shape[0]-debt_consolidation_regression_train_y.sum())/(debt_consolidation_regression_train_y.sum())

In [ ]:
kamil_koc=LogisticRegression(multi_class='ovr').fit(debt_consolidation_regression_x, debt_consolidation_regression_y)
kamil_koc.coef_

In [ ]:
def no_tl_boostrapped(train_data,test_data,base_data_x,base_data_y,n_trials,parameters_path,name,outside_params=None):
    scaler = StandardScaler()
    scaler.fit(base_data_x)
    base_data_x_scaled=scaler.transform(base_data_x)
    base_regression=LogisticRegression(multi_class='ovr').fit(base_data_x_scaled, base_data_y)
    base_mean_vector=np.mean(base_data_x,axis=0)
    base_kde=gaussian_kde(base_data_x.transpose(), bw_method='silverman')
    base_covariance=base_kde.covariance
    base_neg_pos_ratio=(base_data_y.shape[0]-base_data_y.sum())/(base_data_y.sum())
    ##result_dict={}
    #result_dict["name"]=[]
    #result_dict["nrow"]=[]
    #result_dict["y|x_cosine_similarity"]=[]
    #result_dict["x_dist_kl_divergence"]=[]
    #result_dict["0/1_ratio_abs_divergence"]=[]
    for i in range(n_trials):
        #result_dict["name"].append(f"no_tl_boostrapped_{name}_trial_{i}")
        if (name=="wedding") or (name=="renewable_energy"):
            boostrapped_train=resample(train_data, n_samples=int(0.9*train_data.shape[0]), replace=True)
            boostrapped_test=resample(test_data, n_samples=int(0.9*test_data.shape[0]), replace=True)
        else:
            boostrapped_train=resample(train_data, n_samples=int(0.8*train_data.shape[0]), replace=True)
            boostrapped_test=resample(test_data, n_samples=int(0.8*test_data.shape[0]), replace=True)
        y_train,x_train=boostrapped_train['loan_status'],boostrapped_train.drop(['purpose','loan_status'],axis=1)
        y_test,x_test=boostrapped_test['loan_status'],boostrapped_test.drop(['purpose','loan_status'],axis=1)
        scaler = StandardScaler()
        scaler.fit(x_train)
        x_train_scaled=scaler.transform(x_train)
        #result_dict["nrow"].append(x_train.shape[0])
        domain_neg_pos_ratio=(y_train.shape[0]-y_train.sum())/(y_train.sum())
        abs_divergence=abs(base_neg_pos_ratio-domain_neg_pos_ratio)
        #result_dict["0/1_ratio_abs_divergence"].append(abs_divergence)
        domain_regression=LogisticRegression(multi_class='ovr').fit(x_train_scaled, y_train)
        #result_dict["y|x_cosine_similarity"].append(cosine_similarity(domain_regression.coef_,base_regression.coef_))
        domain_mean_vector=np.mean(x_train,axis=0)
        domain_kde=gaussian_kde(x_train.transpose(), bw_method='silverman')
        domain_covariance=domain_kde.covariance
        kl_divergence=kl_mvn(m0=base_mean_vector,S0=base_covariance,m1=domain_mean_vector,S1=domain_covariance)
        #result_dict["x_dist_kl_divergence"].append(kl_divergence)
            # Open the file in binary mode
        with open(parameters_path, 'rb') as file:
        # Call load method to deserialze
            myvar = pickle.load(file)
        if outside_params is not None:
            xgb_params={**myvar.best_params_,**outside_params}
        else:
            xgb_params=myvar.best_params_

        wandb.init(project="transfer learning", entity="otukenotuk",config=myvar.best_params_,name=f"no_tl_boostrapped_{name}_trial_{i}",settings=wandb.Settings(start_method='thread'))
        wandb.summary["nrow"]=x_train.shape[0]
        wandb.summary["y|x_cosine_similarity"]=cosine_similarity(domain_regression.coef_,base_regression.coef_)
        wandb.summary["0/1_ratio_abs_divergence"]=abs_divergence
        wandb.summary["x_dist_kl_divergence"]=kl_divergence
        model=xgb.XGBClassifier(**xgb_params,eval_metric=["auc","logloss","aucpr"],early_stopping_rounds=20)
        model.fit(x_train,
        y_train,
        eval_set=[(x_train,y_train),
        (x_test,y_test)],callbacks=[WandbCallback()])
        wandb.finish()
        with open(f"boostrap_no_tl_models/{name}", 'wb') as file:
        # A new file will be created
            pickle.dump(model, file)
        with open(f"boostrap_data_samples/{name}_train_trial_{i}", 'wb') as file:
        # A new file will be created
            pickle.dump(boostrapped_train, file)
        with open(f"boostrap_data_samples/{name}_test_trial_{i}", 'wb') as file:
        # A new file will be created
            pickle.dump(boostrapped_test, file)
        
    #resutls_df=pd.DataFrame(result_dict)
    #resutls_df.to_csv(f"boostrap_characteristics/no_tl_{name}.csv",index=False)
    #return resutls_df

In [ ]:
no_tl_boostrapped(train_data=credit_card_regression_train,
test_data=credit_card_regression_test,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,parameters_path="results/credit_card_notl_results1.pkl",name="credit_card",outside_params={"n_estimators":200})

In [ ]:
no_tl_boostrapped(train_data=home_improvement_regression_train,
test_data=home_improvement_regression_test,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,parameters_path="results/home_improvement_notl_results1.pkl",name="home_improvement",outside_params={"n_estimators":200})

In [ ]:
no_tl_boostrapped(train_data=other_regression_train,
test_data=other_regression_test,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,parameters_path="results/other_notl_results_deep.pkl",name="other",outside_params={"n_estimators":200})

In [ ]:
no_tl_boostrapped(train_data=major_purchase_regression_train,
test_data=major_purchase_regression_test,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,parameters_path="results/major_purchase_notl_results_deep.pkl",name="major_purchase",outside_params={"n_estimators":200})

In [ ]:
no_tl_boostrapped(train_data=medical_regression_train,
test_data=medical_regression_test,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,parameters_path="results/medical_notl_results_deep.pkl",name="medical",outside_params={"n_estimators":200})

In [ ]:
no_tl_boostrapped(train_data=small_business_regression_train,
test_data=small_business_regression_test,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,parameters_path="results/small_business_notl_results_deep.pkl",name="small_business",outside_params={"n_estimators":200})

In [ ]:
no_tl_boostrapped(train_data=car_regression_train,
test_data=car_regression_test,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,parameters_path="results/car_notl_results_deep.pkl",name="car",outside_params={"n_estimators":200})

In [ ]:
no_tl_boostrapped(train_data=moving_regression_train,
test_data=moving_regression_test,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,parameters_path="results/moving_notl_results_deep.pkl",name="moving",outside_params={"n_estimators":200})

In [ ]:
no_tl_boostrapped(train_data=vacation_regression_train,
test_data=vacation_regression_test,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,parameters_path="results/vacation_notl_results_deep.pkl",name="vacation",outside_params={"n_estimators":200})

In [ ]:
no_tl_boostrapped(train_data=house_regression_train,
test_data=house_regression_test,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,parameters_path="results/house_notl_results_deep.pkl",name="house",outside_params={"n_estimators":200})

In [ ]:
no_tl_boostrapped(train_data=wedding_train,
test_data=wedding_test,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,parameters_path="results/wedding_notl_results_deep.pkl",name="wedding",outside_params={"n_estimators":200})

In [ ]:
no_tl_boostrapped(train_data=renewable_energy_regression_train,
test_data=renewable_energy_regression_test,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,parameters_path="results/renewable_energy_notl_results_deep.pkl",name="renewable_energy",outside_params={"n_estimators":200})

In [ ]:
source_params={'subsample': 1.0,
 'n_estimators': 30,
 'max_depth': 6,
 'eta': 0.14285714285714285,
 'alpha': 0.07999999999999999}

In [ ]:
source_model=xgb.XGBClassifier(**source_params,eval_metric=["auc","logloss","aucpr"],early_stopping_rounds=20)
source_model.fit(debt_consolidation_regression_train_x,
debt_consolidation_regression_train_y,
eval_set=[(debt_consolidation_regression_train_x,debt_consolidation_regression_train_y),
(debt_consolidation_regression_test_x,debt_consolidation_regression_test_y)])

In [ ]:
def tl_no_update_boostrapped(parameters_path,base_model,name,base_data_x,base_data_y,n_trials,outside_params=None):
    scaler = StandardScaler()
    scaler.fit(base_data_x)
    base_data_x_scaled=scaler.transform(base_data_x)
    base_regression=LogisticRegression(multi_class='ovr').fit(base_data_x_scaled, base_data_y)
    base_mean_vector=np.mean(base_data_x,axis=0)
    base_kde=gaussian_kde(base_data_x.transpose(), bw_method='silverman')
    base_covariance=base_kde.covariance
    base_neg_pos_ratio=(base_data_y.shape[0]-base_data_y.sum())/(base_data_y.sum())
    
    for i in range(n_trials):
        with open(f"boostrap_data_samples/{name}_train_trial_{i}", 'rb') as file:
        # Call load method to deserialze
            domain_train = pickle.load(file)
        with open(f"boostrap_data_samples/{name}_test_trial_{i}", 'rb') as file:
        # Call load method to deserialze
            domain_test = pickle.load(file)
        
        y_train,x_train=domain_train['loan_status'],domain_train.drop(['purpose','loan_status'],axis=1)
        y_test,x_test=domain_test['loan_status'],domain_test.drop(['purpose','loan_status'],axis=1)
        x_train_scaled=scaler.transform(x_train)
        # Open the file in binary mode
        scaler = StandardScaler()
        scaler.fit(x_train)
        #result_dict["nrow"].append(x_train.shape[0])
        domain_neg_pos_ratio=(y_train.shape[0]-y_train.sum())/(y_train.sum())
        abs_divergence=abs(base_neg_pos_ratio-domain_neg_pos_ratio)
        #result_dict["0/1_ratio_abs_divergence"].append(abs_divergence)
        domain_regression=LogisticRegression(multi_class='ovr').fit(x_train_scaled, y_train)
        #result_dict["y|x_cosine_similarity"].append(cosine_similarity(domain_regression.coef_,base_regression.coef_))
        domain_mean_vector=np.mean(x_train,axis=0)
        domain_kde=gaussian_kde(x_train.transpose(), bw_method='silverman')
        domain_covariance=domain_kde.covariance
        kl_divergence=kl_mvn(m0=base_mean_vector,S0=base_covariance,m1=domain_mean_vector,S1=domain_covariance)
        
        with open(parameters_path, 'rb') as file:
        # Call load method to deserialze
            myvar = pickle.load(file)

        if outside_params is not None:
            xgb_params={**myvar.best_params_,**outside_params}
        else:
            xgb_params=myvar.best_params_

        wandb.init(project="transfer learning", entity="otukenotuk",config=myvar.best_params_,name=f"tl_no_update_boostrapped_{name}_trial_{i}",settings=wandb.Settings(start_method='thread'))
        wandb.summary["nrow"]=x_train.shape[0]
        wandb.summary["y|x_cosine_similarity"]=cosine_similarity(domain_regression.coef_,base_regression.coef_)
        wandb.summary["0/1_ratio_abs_divergence"]=abs_divergence
        wandb.summary["x_dist_kl_divergence"]=kl_divergence

        model=xgb.XGBClassifier(**xgb_params,eval_metric=["auc","logloss","aucpr"],early_stopping_rounds=20)
        model.fit(x_train,
        y_train,
        eval_set=[(x_train,y_train),
        (x_test,y_test)],callbacks=[WandbCallback()],xgb_model=base_model.get_booster())
        wandb.finish()
        with open(f"boostrap_tl_nonupdate_models/{name}", 'wb') as file:
        # A new file will be created
            pickle.dump(model, file)

In [ ]:
tl_no_update_boostrapped(parameters_path="results_tl_nonupdate/credit_card_tl_model.pkl",base_model=source_model,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,name="credit_card",outside_params={"n_estimators":200})

In [ ]:
tl_no_update_boostrapped(parameters_path="results_tl_nonupdate/home_improvement_tl_model.pkl",base_model=source_model,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,name="home_improvement",outside_params={"n_estimators":200})

In [ ]:
tl_no_update_boostrapped(parameters_path="results_tl_nonupdate/other_tl_model.pkl",base_model=source_model,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,name="other",outside_params={"n_estimators":200})

In [ ]:
tl_no_update_boostrapped(parameters_path="results_tl_nonupdate/major_purchase_tl_model.pkl",base_model=source_model,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,name="major_purchase",outside_params={"n_estimators":200})

In [ ]:
tl_no_update_boostrapped(parameters_path="results_tl_nonupdate/medical_tl_model.pkl",base_model=source_model,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,name="medical",outside_params={"n_estimators":200})

In [ ]:
tl_no_update_boostrapped(parameters_path="results_tl_nonupdate/small_business_tl_model.pkl",base_model=source_model,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,name="small_business",outside_params={"n_estimators":200})

In [ ]:
tl_no_update_boostrapped(parameters_path="results_tl_nonupdate/car_tl_model.pkl",base_model=source_model,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,name="car",outside_params={"n_estimators":200})

In [ ]:
tl_no_update_boostrapped(parameters_path="results_tl_nonupdate/moving_tl_model.pkl",base_model=source_model,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,name="moving",outside_params={"n_estimators":200})

In [ ]:
tl_no_update_boostrapped(parameters_path="results_tl_nonupdate/vacation_tl_model.pkl",base_model=source_model,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,name="vacation",outside_params={"n_estimators":200})

In [ ]:
tl_no_update_boostrapped(parameters_path="results_tl_nonupdate/house_tl_model.pkl",base_model=source_model,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,name="house",outside_params={"n_estimators":200})

In [ ]:
tl_no_update_boostrapped(parameters_path="results_tl_nonupdate/wedding_tl_model.pkl",base_model=source_model,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,name="wedding",outside_params={"n_estimators":200})

In [ ]:
tl_no_update_boostrapped(parameters_path="results_tl_nonupdate/renewable_energy_tl_model.pkl",base_model=source_model,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,name="renewable_energy",outside_params={"n_estimators":200})

In [ ]:
def tl_no_update_boostrapped(parameters_path,base_model,name,base_data_x,base_data_y,n_trials,outside_params=None):
    scaler = StandardScaler()
    scaler.fit(base_data_x)
    base_data_x_scaled=scaler.transform(base_data_x)
    base_regression=LogisticRegression(multi_class='ovr').fit(base_data_x_scaled, base_data_y)
    base_mean_vector=np.mean(base_data_x,axis=0)
    base_kde=gaussian_kde(base_data_x.transpose(), bw_method='silverman')
    base_covariance=base_kde.covariance
    base_neg_pos_ratio=(base_data_y.shape[0]-base_data_y.sum())/(base_data_y.sum())
    
    for i in range(n_trials):
        with open(f"boostrap_data_samples/{name}_train_trial_{i}", 'rb') as file:
        # Call load method to deserialze
            domain_train = pickle.load(file)
        with open(f"boostrap_data_samples/{name}_test_trial_{i}", 'rb') as file:
        # Call load method to deserialze
            domain_test = pickle.load(file)
        
        y_train,x_train=domain_train['loan_status'],domain_train.drop(['purpose','loan_status'],axis=1)
        y_test,x_test=domain_test['loan_status'],domain_test.drop(['purpose','loan_status'],axis=1)
        x_train_scaled=scaler.transform(x_train)
        # Open the file in binary mode
        scaler = StandardScaler()
        scaler.fit(x_train)
        #result_dict["nrow"].append(x_train.shape[0])
        domain_neg_pos_ratio=(y_train.shape[0]-y_train.sum())/(y_train.sum())
        abs_divergence=abs(base_neg_pos_ratio-domain_neg_pos_ratio)
        #result_dict["0/1_ratio_abs_divergence"].append(abs_divergence)
        domain_regression=LogisticRegression(multi_class='ovr').fit(x_train_scaled, y_train)
        #result_dict["y|x_cosine_similarity"].append(cosine_similarity(domain_regression.coef_,base_regression.coef_))
        domain_mean_vector=np.mean(x_train,axis=0)
        domain_kde=gaussian_kde(x_train.transpose(), bw_method='silverman')
        domain_covariance=domain_kde.covariance
        kl_divergence=kl_mvn(m0=base_mean_vector,S0=base_covariance,m1=domain_mean_vector,S1=domain_covariance)
        
        with open(parameters_path, 'rb') as file:
        # Call load method to deserialze
            myvar = pickle.load(file)

        if outside_params is not None:
            xgb_params={**myvar.best_params_,**outside_params}
        else:
            xgb_params=myvar.best_params_

        wandb.init(project="transfer learning", entity="otukenotuk",config=myvar.best_params_,name=f"tl_no_update_boostrapped_{name}_trial_{i}",settings=wandb.Settings(start_method='thread'))
        wandb.summary["nrow"]=x_train.shape[0]
        wandb.summary["y|x_cosine_similarity"]=cosine_similarity(domain_regression.coef_,base_regression.coef_)
        wandb.summary["0/1_ratio_abs_divergence"]=abs_divergence
        wandb.summary["x_dist_kl_divergence"]=kl_divergence

        model=xgb.XGBClassifier(**xgb_params,eval_metric=["auc","logloss","aucpr"],early_stopping_rounds=20)
        model.fit(x_train,
        y_train,
        eval_set=[(x_train,y_train),
        (x_test,y_test)],callbacks=[WandbCallback()],xgb_model=base_model.get_booster())
        wandb.finish()
        with open(f"boostrap_tl_nonupdate_models/{name}", 'wb') as file:
        # A new file will be created
            pickle.dump(model, file)

In [ ]:
source_tl={'subsample': 1.0,
 'max_depth': 6,
 'eta': 0.14285714285714285,
 'alpha': 0.07999999999999999}

In [ ]:
def multiround_revising(num_rounds,source_params,x_source,y_source,x_target,y_target,min_split_loss=1,min_child_weight=5):
    dtrain_target=xgb.DMatrix(x_target, y_target)
    for i in range(num_rounds):
        if i==0:
   
            source_model=xgb.XGBClassifier(**source_params,n_estimators=1)
            source_model.fit(x_source,y_source)

        else:

            source_model.fit(x_source,y_source,xgb_model=pruned_trees)

        refreshed_trees=xgb.train({"process_type": "update", "updater": "refresh", "refresh_leaf": True},
        dtrain=dtrain_target,num_boost_round=i+1,xgb_model=source_model.get_booster())

        pruned_trees = xgb.train(
        {"process_type": "update", "updater": "prune","min_split_loss":min_split_loss,"min_child_weight":min_child_weight},
        dtrain=dtrain_target,
        num_boost_round=i+1,
        xgb_model=refreshed_trees)       
    return pruned_trees

In [ ]:
def tl_update_boostrapped(name,n_trials,parameters_path,base_data_x,base_data_y,min_split_loss,min_child_weight,source_tl,outside_params=None,):
    scaler = StandardScaler()
    scaler.fit(base_data_x)
    base_data_x_scaled=scaler.transform(base_data_x)
    base_regression=LogisticRegression(multi_class='ovr').fit(base_data_x_scaled, base_data_y)
    base_mean_vector=np.mean(base_data_x,axis=0)
    base_kde=gaussian_kde(base_data_x.transpose(), bw_method='silverman')
    base_covariance=base_kde.covariance
    base_neg_pos_ratio=(base_data_y.shape[0]-base_data_y.sum())/(base_data_y.sum())
    for i in range(n_trials):
        with open(f"boostrap_data_samples/{name}_train_trial_{i}", 'rb') as file:
        # Call load method to deserialze
            domain_train = pickle.load(file)
        with open(f"boostrap_data_samples/{name}_test_trial_{i}", 'rb') as file:
        # Call load method to deserialze
            domain_test = pickle.load(file)
        
        y_train,x_train=domain_train['loan_status'],domain_train.drop(['purpose','loan_status'],axis=1)
        y_test,x_test=domain_test['loan_status'],domain_test.drop(['purpose','loan_status'],axis=1)
        scaler = StandardScaler()
        scaler.fit(x_train)
        x_train_scaled=scaler.transform(x_train)
        domain_neg_pos_ratio=(y_train.shape[0]-y_train.sum())/(y_train.sum())
        abs_divergence=abs(base_neg_pos_ratio-domain_neg_pos_ratio)
        #result_dict["0/1_ratio_abs_divergence"].append(abs_divergence)
        domain_regression=LogisticRegression(multi_class='ovr').fit(x_train_scaled, y_train)
        #result_dict["y|x_cosine_similarity"].append(cosine_similarity(domain_regression.coef_,base_regression.coef_))
        domain_mean_vector=np.mean(x_train,axis=0)
        domain_kde=gaussian_kde(x_train.transpose(), bw_method='silverman')
        domain_covariance=domain_kde.covariance
        kl_divergence=kl_mvn(m0=base_mean_vector,S0=base_covariance,m1=domain_mean_vector,S1=domain_covariance)
        
        # Open the file in binary mode
        with open(parameters_path, 'rb') as file:
            # Call load method to deserialze
            myvar = pickle.load(file)

        if outside_params is not None:
            xgb_params={**myvar.best_params_,**outside_params}
        else:
            xgb_params=myvar.best_params_

        wandb.init(project="transfer learning", entity="otukenotuk",config=myvar.best_params_,name=f"tl_update_boostrapped_{name}_trial_{i}",settings=wandb.Settings(start_method='thread'))
        wandb.summary["nrow"]=x_train.shape[0]
        wandb.summary["y|x_cosine_similarity"]=cosine_similarity(domain_regression.coef_,base_regression.coef_)
        wandb.summary["0/1_ratio_abs_divergence"]=abs_divergence
        wandb.summary["x_dist_kl_divergence"]=kl_divergence

        updated_source=multiround_revising(num_rounds=30,source_params=source_tl,x_source=base_data_x,y_source=base_data_y,x_target=x_train,y_target=y_train,min_split_loss=min_split_loss,min_child_weight=min_child_weight)
        model=xgb.XGBClassifier(**xgb_params,eval_metric=["auc","logloss","aucpr"],early_stopping_rounds=20)
        model.fit(x_train,
        y_train,
        eval_set=[(x_train,y_train),
        (x_test,y_test)],callbacks=[WandbCallback()],xgb_model=updated_source)
        wandb.finish()
        with open(f"boostrap_tl_update_models/{name}", 'wb') as file:
        # A new file will be created
            pickle.dump(model, file)

In [ ]:
tl_update_boostrapped(parameters_path="results_tl_update/credit_card_tl_model.pkl",min_split_loss=1.5,
min_child_weight=1,source_tl=source_tl,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,name="credit_card",outside_params={"n_estimators":200})
        

In [ ]:
tl_update_boostrapped(parameters_path="results_tl_update/home_improvement_tl_model.pkl",min_split_loss=0.3,
min_child_weight=1,source_tl=source_tl,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,name="home_improvement",outside_params={"n_estimators":200})

In [ ]:
tl_update_boostrapped(parameters_path="results_tl_update/other_tl_model.pkl",min_split_loss=0.3,
min_child_weight=1,source_tl=source_tl,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,name="other",outside_params={"n_estimators":200})

In [ ]:
tl_update_boostrapped(parameters_path="results_tl_update/major_purchase_tl_model.pkl",min_split_loss=2,
min_child_weight=1,source_tl=source_tl,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,name="major_purchase",outside_params={"n_estimators":200})

In [ ]:
tl_update_boostrapped(parameters_path="results_tl_update/medical_tl_model.pkl",min_split_loss=4,
min_child_weight=1,source_tl=source_tl,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,name="medical",outside_params={"n_estimators":200})

In [ ]:
tl_update_boostrapped(parameters_path="results_tl_update/small_business_tl_model.pkl",min_split_loss=0.0001,
min_child_weight=1,source_tl=source_tl,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,name="small_business",outside_params={"n_estimators":200})

In [ ]:
tl_update_boostrapped(parameters_path="results_tl_update/car_tl_model.pkl",min_split_loss=0.03,
min_child_weight=1,source_tl=source_tl,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,name="car",outside_params={"n_estimators":200})

In [ ]:
tl_update_boostrapped(parameters_path="results_tl_update/moving_tl_model.pkl",min_split_loss=0.001,
min_child_weight=2,source_tl=source_tl,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,name="moving",outside_params={"n_estimators":200})

In [ ]:
tl_update_boostrapped(parameters_path="results_tl_update/vacation_tl_model.pkl",min_split_loss=1,
min_child_weight=2,source_tl=source_tl,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,name="vacation",outside_params={"n_estimators":200})

In [ ]:
tl_update_boostrapped(parameters_path="results_tl_update/house_tl_model.pkl",min_split_loss=0.00000001,
min_child_weight=1,source_tl=source_tl,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,name="house",outside_params={"n_estimators":200})

In [ ]:
tl_update_boostrapped(parameters_path="results_tl_update/wedding_tl_model.pkl",min_split_loss=0.3,
min_child_weight=1,source_tl=source_tl,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,name="wedding",outside_params={"n_estimators":200})

In [ ]:
tl_update_boostrapped(parameters_path="results_tl_update/renewable_energy_tl_model.pkl",min_split_loss=0.1,
min_child_weight=1,source_tl=source_tl,
base_data_x=debt_consolidation_regression_x,
base_data_y=debt_consolidation_regression_y,
n_trials=5,name="renewable_energy",outside_params={"n_estimators":200})